From government data I get data in ascii-grid format which I need to transform into latitude/longitude data.
Using the data I can determine the lowest pollution rate.
This information is available on https://geodata.rivm.nl/gcn/. I will use data of 2018.

# Using cell size of 3 km x 3 km
The PM25 data has a shape of 320*280 = 89600 cells.
Each cell represents 1 km x 1 km

89600 is too much to handle using Nominatim to get address details.
Nominatim limits the number of calls to max 1 call per second.
I set the sleep timer to 2 seconds to be sure I am within limits.

I will change the cell size to 3 km x 3 km which still gives good results for PM25 data.
Thereby reducing the number of cells to 4886.

In [1]:
# Generic imports

In [2]:
import pandas as pd
import time as time
import numpy as np

# Retrieve Pollution Rate Data from Government site
The dataset is downloaded from https://geodata.rivm.nl/gcn/ in a ASCII-GRID format. 
See also https://en.wikipedia.org/wiki/Esri_grid

Location in grid is according to the 'Amersfoortse coordinaten' which is described in dutch on
https://nl.wikipedia.org/wiki/Rijksdriehoeksco%C3%B6rdinaten
Amersfoort has a location of X=155000 and Y=463000

First 6 lines show
* NCOLS          280  (nr of cells in X direction)
* NROWS          320  (nr of cells in Y direction)
* XLLCORNER        0  (X coordinate)
* YLLCORNER       300000  Y Coordinate
* CELLSIZE         1000  (1000 meters so 1 km)
* NODATA_VALUE  -0.9990E+03 (this is the value if there is no data)


To convert the data into longitude and latitude I use code from https://thomasv.nl/2014/03/rd-naar-gps/
This converts the so called Rijksdriehoek coordinates into latitude and longitude. 
Code is a separate class called RDWGSConverter


In [3]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="test1")
from RDWGSConverter import RDWGSConverter  # import module to convert Rijksdriehoek coordinates into latitude and longitude


# Load Pollution Data into a Dataframe

In [4]:
listOfColumnWidth = [13] * 280  # 280 columns of width 13
df_pm25 = pd.read_fwf('conc_pm25_2018/conc_pm25_2018.asc', widths=listOfColumnWidth,skiprows=6,header=None)

#### Replace missing values in the data(which is represented as -999.0) by nan

In [5]:
df_pm25.replace(-999.0, np.nan,inplace=True)
df_pm25.describe()

,0,1,2,3,4,5,6,7,8,9,...,270,271,272,273,274,275,276,277,278,279
count,0.0,0.0,0.0,0.0,0.0,0.0,2.000000,4.000000,6.000000,9.000000,...,54.000000,53.000000,51.000000,49.000000,47.000000,46.000000,32.000000,13.000000,1.000,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN,9.634000,9.626750,9.614167,9.595444,...,8.407759,8.416208,8.476588,8.506490,8.513638,8.490870,8.541688,8.576846,8.418,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,0.007071,0.009777,0.012734,0.014028,...,0.331099,0.336813,0.341648,0.379752,0.380547,0.371899,0.336820,0.241870,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,9.629000,9.616000,9.596000,9.573000,...,7.751000,7.762000,7.785000,7.801000,7.820000,7.843000,7.879000,8.252000,8.418,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,9.631500,9.619750,9.605500,9.584000,...,8.322000,8.222000,8.280500,8.280000,8.281000,8.269750,8.352000,8.316000,8.418,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,9.634000,9.627500,9.616500,9.598000,...,8.459000,8.498000,8.617000,8.602000,8.568000,8.496500,8.629000,8.723000,8.418,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,9.636500,9.634500,9.624500,9.608000,...,8.650250,8.651000,8.723000,8.801000,8.820500,8.834750,8.824000,8.783000,8.418,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,9.639000,9.636000,9.627000,9.611000,...,8.917000,8.866000,8.855000,9.016000,8.995000,8.982000,8.928000,8.836000,8.418,NaN


# Convert grid data into a single column and determine x and y values

The matrix of 320*280 needs to be converted to a list with 3 columns
latitude, longitude and the value of the relevant cell in the matrix
So I need to read in a loop 320*280 and based on that I can determine the X and Y value in Rijkscoordinaten
and then convert it into latitude,longitude.
Only keep cells which do not contain NAN.

Also reduce the number of cells used by using a cell size of 3 km x 3 km.
This will be achieved by stepping through the grid with a step of 3.



In [6]:

conv = RDWGSConverter()

area_size=3 # set to 3 km. A cell in the loaded grid is (1 KM)

# netherlands is 320km high by 280 wide
# nr of rows is 320. so it loops over rows which is y range !!!
# A row number starts with 0 and the increments
# The Y coordinate starts at 620 and decrements.
# So row 0 is y-coordinate 620
# y_top is used to determine correct position in the grid
# Amsterdam is on rijkscoordinaat 487432 in Y direction
# This needs to convert to 487432/1000 = apx 487
# then position in the grid is on 620-487=133
# Coordinates are represented in meters. So I have to multiply a grid position by 1000 meters since a cell is 1 km.

# den helder is x,y 110464, 544944. So apx 110,544 to determine grid cell in the data
# amsterdam is in the area of  119648, 487432 so apx 119,487 to determine grid cell in the data
# I use a bigger area a but above Den Helder and below Amsterdam which roughly is North-Holland
y_top = 620  # this is used to calculate correct position in the grid
df_limit = df_pm25

#x_range=range(0,df_limit.shape[1],area_size)   
#y_range=range(0,df_limit.shape[0],area_size) # loop from highest point to lowest point
x_range=range(100,130,area_size)   
y_range=range(y_top - 555,y_top-480,area_size) # loop from highest point to lowest point

df1 = pd.DataFrame()
k = 0
for rd_y in y_range:
    for rd_x in x_range:
        if pd.notnull(df_limit.iat[rd_y,rd_x]): 
            #print(i,j,df_limit.iat[i,j])
            df1.loc[k,'pm25'] = df_limit.iat[rd_y,rd_x]
            
            wgsCoords = conv.fromRdToWgs( [rd_x*1000,(y_top-rd_y)*1000] )
            #print(i,j,k,wgsCoords)
            df1.loc[k,'lat'] =  wgsCoords[0]
            df1.loc[k,'long'] = wgsCoords[1]
            df1.loc[k,'rd_x'] = rd_x*1000
            df1.loc[k,'rd_y'] = (y_top-rd_y)*1000
            k = k + 1
df1.shape            

(241, 5)

In [7]:
df1.head()


,pm25,lat,long,rd_x,rd_y
0,8.809,52.979188,4.568342,100000.0,555000.0
1,8.909,52.979485,4.613003,103000.0,555000.0
2,9.107,52.979766,4.657665,106000.0,555000.0
3,8.971,52.980030,4.702328,109000.0,555000.0
4,8.907,52.980278,4.746992,112000.0,555000.0


In [8]:
# A city is not always returned. sometimes it is village, town etc.
# I will only use data from real cities
# A valid location can be a city, town or village
# display_name is always there (combiningy country,city,village etc)
# So for this assignment diplay_name is ok.

for i in range(df1.shape[0]):
    #print(i)
    #wgsCoords = conv.fromRdToWgs( [110000,i] )
    wgsCoords = [df1.iat[i,1],df1.iat[i,2]]
    #print(wgsCoords)
    location = geolocator.reverse(wgsCoords,timeout=60) 
    # I use for location the display_name
    # places can be suburb,town,city,village and hamlets. Not all these identiers are always in JSON
    # display_name is always there
    try:
        #df1.loc[i,'suburb'] = location.raw['address']['suburb']
        df1.loc[i,'location'] = location.raw['display_name']
        print(i,'location:',location.raw['display_name'])
        #print(i,'city:',location.raw['address']['city'],',',location.raw['address']['suburb'])
        #df1.loc[i,'city'] = location.raw['address']['city']
    except:
        print(location.raw)     
        pass
    time.sleep(2) # set sleep to 2 seconds to prevent error: 429 too may requests


0 location: Nederland
1 location: Nederland
2 location: Den Hoorn, Texel, Noord-Holland, Nederland
3 location: Den Hoorn, Texel, Noord-Holland, Nederland
4 location: Den Hoorn, Texel, Noord-Holland, Nederland
5 location: Den Hoorn, Texel, Noord-Holland, Nederland
6 location: Den Helder, Noord-Holland, Nederland
7 location: Den Helder, Noord-Holland, Nederland
8 location: Den Oever, Hollands Kroon, Noord-Holland, Nederland
9 location: Den Oever, Hollands Kroon, Noord-Holland, Nederland
10 location: Nederland
11 location: Nederland
12 location: Den Hoorn, Texel, Noord-Holland, Nederland
13 location: Den Helder, Noord-Holland, Nederland
14 location: Marsdiepstraat, Den Helder, Noord-Holland, Nederland, 1782DM, Nederland
15 location: Den Helder, Noord-Holland, Nederland, 1781AD, Nederland
16 location: Den Helder, Noord-Holland, Nederland
17 location: Den Helder, Noord-Holland, Nederland
18 location: Den Oever, Hollands Kroon, Noord-Holland, Nederland
19 location: Den Oever, Hollands Kroon,

109 location: Groetpad, Koedijk, Langedijk, Noord-Holland, Nederland, 1831EP, Nederland
110 location: Amelsgroet, Zuid-Scharwoude, Langedijk, Noord-Holland, Nederland, 1722KW, Nederland
111 location: Kamerlingh Onnesweg, Bedrijventerrein, Heerhugowaard, Noord-Holland, Nederland, 1704RK, Nederland
112 location: 12, Berkmeerdijk, Buitengebied Noord, Obdam, Heerhugowaard, Noord-Holland, Nederland, 1713KW, Nederland
113 location: 10, Kaag, Spanbroek, Opmeer, Noord-Holland, Nederland, 1715KR, Nederland
114 location: 't War, Spanbroek, Opmeer, Noord-Holland, Nederland, 1715GX, Nederland
115 location: Bergen aan Zee, Bergen (NH), Noord-Holland, Nederland
116 location: Voert, Bergen-Binnen, Bergen (NH), Noord-Holland, Nederland, 1861PE, Nederland
117 location: 18, Nesdijk, Bergen-Binnen, Bergen (NH), Noord-Holland, Nederland, 1861MK, Nederland
118 location: 13, Zeepziederstraat, Alkmaar, Noord-Holland, Nederland, 1825GK, Nederland
119 location: Outside Living Industries, 87, Berenkoog, Beverko

195 location: 3, Nauernasche Vaartdijk, Westzaan, Zaanstad, Noord-Holland, Nederland, 1551PR, Nederland
196 location: Westerwindpad, Zaandam, Zaanstad, Noord-Holland, Nederland, 1507WT, Nederland
197 location: Twiskeweg, Zaandam, Zaanstad, Noord-Holland, Nederland, 1503AH, Nederland
198 location: De Zuiderlaaik, Oostzaan, Noord-Holland, Nederland, 1511GW, Nederland
199 location: Ilperveld, Dorre Ilp, Landsmeer, Noord-Holland, Nederland, 1127PW, Nederland
200 location: Noordmeer, Broek in Waterland, Waterland, Noord-Holland, Nederland, 1151CW, Nederland
201 location: KD-30, Stroweg, Overveen, Bloemendaal, Noord-Holland, Nederland, 2051EA, Nederland
202 location: Euphrasia, Dennenweg, Veen en Duin, Bloemendaal, Noord-Holland, Nederland, 2061HS, Nederland
203 location: Hekslootpolder - Vogelkijkhut, Spaarndamseweg, Spaarndam-West, Spaarndam gem. Haarlem, Haarlem, Noord-Holland, Nederland, 2063, Nederland
204 location: Ringweg, Spaarndam-Oost, Spaarndam, Haarlemmermeer, Noord-Holland, Nede

http://www.nationaalgeoregister.nl/geonetwork/srv/dut/catalog.search#/metadata/29c17585-e702-463f-a5dc-99d34b17d333


In [9]:
df_pm25.shape

(320, 280)

In [10]:
df_pm25.loc[117:121,163:170]

,163,164,165,166,167,168,169,170
117,9.813,9.602,9.565,9.575,9.720,9.620,9.630,9.752
118,9.749,9.642,9.568,9.945,10.050,9.700,9.749,9.634
119,9.832,9.587,9.445,9.629,9.648,9.662,9.661,9.558
120,9.708,9.444,9.459,9.651,9.594,9.694,9.715,9.686
121,9.659,9.643,9.744,9.751,9.654,9.748,9.907,9.832


In [11]:
df1.tail(30)

,pm25,lat,long,rd_x,rd_y,location
211,10.44,52.386100,4.579320,100000.0,489000.0,"Meertje van Burdet, Visscherspad, Overveen, Bl..."
212,12.71,52.386393,4.623382,103000.0,489000.0,"2D-0031, Garenkokerskade, Haarlem, Noord-Holla..."
213,11.79,52.386669,4.667446,106000.0,489000.0,"Busweg, Waarder-en Veerpolder, Haarlem, Noord-..."
214,11.31,52.386928,4.711510,109000.0,489000.0,"Vinkebrug, Halfweg, Haarlemmermeer, Noord-Holl..."
215,11.56,52.387172,4.755574,112000.0,489000.0,"99, Wethouder Rijkeboerweg, Halfweg, Haarlemme..."
216,12.08,52.387398,4.799639,115000.0,489000.0,"Australiëhavenweg, Amsterdam, Noord-Holland, N..."
217,13.56,52.387609,4.843705,118000.0,489000.0,"Ringweg-West, Sloterdijk, Amsterdam, Noord-Hol..."
218,13.18,52.387803,4.887771,121000.0,489000.0,"78, Vierwindenstraat, Centrum, Amsterdam, Noor..."
219,11.53,52.387980,4.931838,124000.0,489000.0,"G.T. Ketjenweg, Nieuwendammerham, Amsterdam, N..."
220,10.98,52.388142,4.975904,127000.0,489000.0,"Weerslootpad, Amsterdam, Noord-Holland, Nederl..."


In [12]:
df1.describe()

,pm25,lat,long,rd_x,rd_y
count,241.000000,241.000000,241.000000,241.000000,241.000000
mean,10.112241,52.655296,4.781341,114004.149378,518825.726141
std,1.183610,0.197320,0.123855,8381.774835,21956.081209
min,8.019000,52.332179,4.568342,100000.000000,483000.000000
25%,9.067000,52.493941,4.666130,106000.000000,501000.000000
50%,9.995000,52.656776,4.793126,115000.000000,519000.000000
75%,11.030000,52.819350,4.885328,121000.000000,537000.000000
max,14.400000,52.981265,4.976404,127000.000000,555000.000000


In [13]:
df1.to_csv(r'df1-North-Holland-3-by-3-km.csv')

In [14]:
#df_pm25.to_csv(r'df_pm25.csv')